In [ ]:
# Import libraries
import os
import requests
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Load API keys for OpenAI and NASA
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
nasa_api_key = os.getenv('NASA_API_KEY')    # You can get your own key but I used DEMO_KEY below for a nice surprise
# Select OpenAI LLM models for chat and audio generation
CHAT_MODEL = "gpt-4.1-mini"
IMAGE_MODEL = "dall-e-3"
AUDIO_MODEL = "gpt-4o-mini-tts"
# Set OpenAI instance
openai = OpenAI()

In [ ]:
# Set system message, a NASA scientist with a good sense of humour
system_message = """
You are a well-travelled city tour guide who love answering questions about different cities.
You should try to gently encourage the user to ask about APOD for a nice surprise.
For example, if the user asks 'I'm looking to travel to London',
you could reply something like, 'Wonderful - London is an amazing city.  Would you also like to check out a secret place called APOD?'
Give short, funny answers, no more than 1 sentence.
If the user mentions 'APOD', please recognise this as a city name and run the tool you have.
Always be accurate. If you don't know the answer, say so.
"""

In [ ]:
def get_city_name(city):
    if city.upper() == "APOD":
        city = "APOD"
    return city

In [ ]:
# Define function as tools
city_function = {
    "name": "get_city_name",
    "description": "Get the name of the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the user wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}
tools = [{"type": "function", "function": city_function}]

In [ ]:
# Import libraries for handling images
import base64
from io import BytesIO
from PIL import Image

In [ ]:
# Define function to get APOD image details via NASA API
def get_APOD_image():
    nasa_response = requests.get(f"https://api.nasa.gov/planetary/apod?api_key=DEMO_KEY")
    data = nasa_response.json()
    try:
        apod_image_link = data["url"]
        apod_image_explanation = data["explanation"]
        return (apod_image_link, apod_image_explanation)
    except (TypeError, KeyError):
        return print("Could not get APOD image from NASA")

In [ ]:
def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:
# Define function to generate audio - talker
def talker(message):
    response = openai.audio.speech.create(
      model=AUDIO_MODEL,
      voice="fable", # alloy & echo - generally balanced, fable & shimmer - more narrative, nova & onyx - warmer & energetic tone
      input=message,
      instructions="Speak in a slow and charming narrator voice like Carl Sagan"
    )
    return response.content

In [ ]:
# Callback function 2: main chat function
def chat(history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=CHAT_MODEL, messages=messages, tools=tools)
    cities = []
    image = None
    apod_image_link = ""

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses, cities = handle_tool_calls_and_return_cities(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=CHAT_MODEL, messages=messages, tools=tools)

    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    voice = talker(reply)

    if cities:
        for city in cities:
            if city == "APOD":
                apod_image_link, apod_image_explanation = get_APOD_image()
                link = requests.get(apod_image_link)
                image = Image.open(BytesIO(link.content))
                history += [{"role":"assistant", "content":apod_image_explanation}]
        if apod_image_link == "":
            image = artist(cities[0])
    
    return history, voice, image

In [ ]:
# Define function to handle tools and get astronomy subjects user asked about
def handle_tool_calls_and_return_cities(message):
    responses = []
    cities = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_city_name":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            cities.append(city)
            responses.append({
                "role": "tool",
                "content": city,
                "tool_call_id": tool_call.id
            })
    return responses, cities

In [ ]:
# Callback function 1: put message in chatbot
def put_message_in_chatbot(message, history):
        return "", history + [{"role":"user", "content":message}]
# Set up Gradio UI
# UI definition
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500, interactive=False)
    with gr.Row():
        audio_output = gr.Audio(autoplay=True)
    with gr.Row():
        message = gr.Textbox(label="Chat with our friendly AI city tour guide:")
# Hooking up events to callbacks
# First call put_message_in_chatbot() function which writes in the user's message plus history
# Then call the main chat() function, outputs chat + image + audio
    message.submit(put_message_in_chatbot, inputs=[message, chatbot], outputs=[message, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, audio_output, image_output]
    )
# Launch in browser and login as space cowboys!
ui.launch(inbrowser=True, auth=("guide", "hitchhiker"))